<a href="https://colab.research.google.com/github/anioki/ML_training_projects/blob/v2/spam/SPAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import tensorflow as tf
from tensorflow.keras import layers, models

#Download data

In [2]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                           title                                              size  lastUpdated          downloadCount  
------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                   Reddit Vaccine Myths                              232KB  2021-06-30 11:37:00           8580  
crowww/a-large-scale-fish-dataset                             A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           5233  
imsparsh/musicnet-dataset                                     MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1835  
promptcloud/careerbuilder-job-listing-2020                    Careerbuilder Job Listing 2020                     42MB  2021-03-05 06:59:52           1208  
coloradokb/dandelionimages                                    Da

In [5]:
! kaggle datasets download -d uciml/sms-spam-collection-dataset

  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 106MB/s]


In [6]:
! mkdir data
! unzip sms-spam-collection-dataset.zip -d data

Archive:  sms-spam-collection-dataset.zip
  inflating: data/spam.csv           


In [7]:
df = pd.read_csv('/content/data/spam.csv', encoding="ISO-8859-1")
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


#Data Preprocessing

In [8]:
df['v1'] = df['v1'].replace(['ham'], 0)
df['v1'] = df['v1'].replace(['spam'], 1)
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [9]:
X = df['v2']
y = df['v1']

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [11]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=100)

In [12]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen=100)

#Model

In [13]:
model = models.Sequential()
model.add(layers.InputLayer(input_shape=[100]))
model.add(layers.Embedding(1000, 50, input_length=100))
model.add(layers.LSTM(32))
model.add(layers.Dense(256,name='FC1',activation='relu'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(rate = 0.5))
model.add(layers.Dense(1,name='out_layer'))
model.add(layers.Activation('sigmoid')) 

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 50)           50000     
_________________________________________________________________
lstm (LSTM)                  (None, 32)                10624     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               8448      
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0

In [14]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [15]:
model.fit(sequences_matrix, Y_train, batch_size=128, epochs=5, validation_split=0.2)

Epoch 1/5
28/28 [==============================] - 4s 64ms/step - loss: 0.3322 - accuracy: 0.8741 - val_loss: 0.1416 - val_accuracy: 0.9619
Epoch 2/5
28/28 [==============================] - 1s 49ms/step - loss: 0.1071 - accuracy: 0.9767 - val_loss: 0.0720 - val_accuracy: 0.9877
Epoch 3/5
28/28 [==============================] - 1s 49ms/step - loss: 0.0481 - accuracy: 0.9879 - val_loss: 0.0595 - val_accuracy: 0.9865
Epoch 4/5
28/28 [==============================] - 1s 48ms/step - loss: 0.0316 - accuracy: 0.9910 - val_loss: 0.0616 - val_accuracy: 0.9854
Epoch 5/5
28/28 [==============================] - 1s 49ms/step - loss: 0.0252 - accuracy: 0.9924 - val_loss: 0.0750 - val_accuracy: 0.9809


In [16]:
predict_result = model.evaluate(test_sequences_matrix,Y_test)

35/35 [==============================] - 0s 7ms/step - loss: 0.0507 - accuracy: 0.9848
